In [1]:
import time
import random
import torch
import pandas as pd
from sklearn.model_selection import ParameterSampler
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import DataLoader, Dataset
from transformers import BertModel, AdamW
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")
feature  =['ACK Flag Cnt',
 'Active Max',
 'Active Mean',
 'Active Std',
 'Bwd Header Len',
 'Bwd IAT Min',
 'Bwd IAT Tot',
 'Bwd Pkt Len Max',
 'Bwd Pkt Len Mean',
 'Bwd Pkt Len Min',
 'FIN Flag Cnt',
 'Flow Byts/s',
 'Flow Duration',
 'Flow IAT Max',
 'Flow IAT Mean',
 'Flow IAT Std',
 'Fwd IAT Mean',
 'Fwd IAT Min',
 'Fwd IAT Std',
 'Fwd IAT Tot',
 'Fwd Pkt Len Max',
 'Fwd Pkt Len Std',
 'Fwd Pkts/s',
 'Idle Max',
 'Idle Mean',
 'Idle Std',
 'Init Bwd Win Byts',
 'Pkt Len Max',
 'Pkt Len Min',
 'Pkt Len Var',
 'Pkt Size Avg',
 'Protocol',
 'Src Port',
 'Subflow Bwd Byts',
 'Tot Fwd Pkts',
 'TotLen Bwd Pkts',
 'Label']


# Define the Transformer-based model
class IoTClassifier(torch.nn.Module):
    def __init__(self, input_dim, num_classes):
        super(IoTClassifier, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, 768)
        self.transformer = BertModel.from_pretrained('bert-base-uncased')
        self.fc2 = torch.nn.Linear(768, num_classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.transformer(inputs_embeds=x.unsqueeze(1)).last_hidden_state
        x = torch.mean(x, dim=1)  # Pooling
        x = self.fc2(x)
        return x

# Dataset class
class IoTDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Load data
train = pd.read_csv('./small/AD-S1.csv', usecols=feature)
test = pd.read_csv('./small/AD-S2.csv', usecols=feature)
train = train.replace(-9999, 0).fillna(0)
test = test.replace(-9999, 0).fillna(0)

X_train = train.drop(['Label'], axis=1)
y_train = train['Label']
X_test = test.drop(['Label'], axis=1)
y_test = test['Label']

# Encode labels and scale features
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Hyperparameters to search over
param_grid = {
    'batch_size': [16, 32, 64],
    'learning_rate': [1e-5, 3e-5, 5e-5],
    'epochs': [3, 5, 7],
    'weight_decay': [0, 0.01, 0.1]
}
param_list = list(ParameterSampler(param_grid, n_iter=5, random_state=42))

# Train function with hyperparameters
def train_and_evaluate(params):
    print(f"Training with params: {params}")
    batch_size = params['batch_size']
    learning_rate = params['learning_rate']
    epochs = params['epochs']
    weight_decay = params['weight_decay']

    # Prepare DataLoader
    train_dataset = IoTDataset(X_train, y_train)
    test_dataset = IoTDataset(X_test, y_test)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    # Initialize model, loss, optimizer
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = IoTClassifier(X_train.shape[1], len(label_encoder.classes_)).to(device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # Training loop
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)

            # Forward pass
            outputs = model(features)
            loss = criterion(outputs, labels)

            # Backpropagation and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {total_loss / len(train_loader):.4f}')

    # Evaluation
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for features, labels in test_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            _, predicted = torch.max(outputs, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    f1 = f1_score(all_labels, all_preds, average='macro')
    print(f'F1-Score: {f1:.4f}')
    return f1

# Perform Random Search
best_f1 = 0
best_params = None

for params in param_list:
    f1 = train_and_evaluate(params)
    if f1 > best_f1:
        best_f1 = f1
        best_params = params

print(f'Best F1-Score: {best_f1:.4f} with params: {best_params}')


Training with params: {'weight_decay': 0, 'learning_rate': 3e-05, 'epochs': 3, 'batch_size': 32}
Epoch [1/3], Loss: 0.7851
Epoch [2/3], Loss: 0.4618
Epoch [3/3], Loss: 0.3942
F1-Score: 0.7836
Training with params: {'weight_decay': 0, 'learning_rate': 1e-05, 'epochs': 3, 'batch_size': 16}
Epoch [1/3], Loss: 0.9119
Epoch [2/3], Loss: 0.4968
Epoch [3/3], Loss: 0.4123
F1-Score: 0.7850
Training with params: {'weight_decay': 0.01, 'learning_rate': 3e-05, 'epochs': 7, 'batch_size': 16}
Epoch [1/7], Loss: 0.7640
Epoch [2/7], Loss: 0.4707
Epoch [3/7], Loss: 0.4168
Epoch [4/7], Loss: 0.3821
Epoch [5/7], Loss: 0.3581
Epoch [6/7], Loss: 0.3472
Epoch [7/7], Loss: 0.3204
F1-Score: 0.7777
Training with params: {'weight_decay': 0.01, 'learning_rate': 3e-05, 'epochs': 3, 'batch_size': 32}
Epoch [1/3], Loss: 0.8005
Epoch [2/3], Loss: 0.4669
Epoch [3/3], Loss: 0.3986
F1-Score: 0.8097
Training with params: {'weight_decay': 0, 'learning_rate': 1e-05, 'epochs': 7, 'batch_size': 16}
Epoch [1/7], Loss: 0.9011

In [2]:
best_params

{'weight_decay': 0.01, 'learning_rate': 3e-05, 'epochs': 3, 'batch_size': 32}